In [2]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging
import numpy as np
import plotly.graph_objects as go
from PIL import Image

import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.autograd import Variable

from torch.optim import Adam, SGD
import gc

from ResNet_CNN import *

import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

from torch.optim import Adam
from torch import nn
import torchvision
from functools import partial
from collections import OrderedDict

import torchvision.models as models

from torchsummary import summary

In [3]:
model = res_conv1(1,64)

In [4]:
model.load_state_dict(torch.load('./cnn_res.pth'))

<All keys matched successfully>

In [5]:
loaded = np.load('images_npy/EBAY/EBAY_0.npz')

In [6]:
x = loaded['x']
y = loaded['y']

In [7]:
x.shape, y.shape

((256, 256, 1000), (1000,))

In [8]:
X = x.reshape((-1, 256, 256)).astype(np.float32)
Y = y.reshape((-1, 1)).astype(np.float32)

In [9]:
X.shape

(1000, 256, 256)

In [10]:
test_X = X.reshape(-1, 1, 256, 256)
test_X = torch.from_numpy(test_X)
test_Y = torch.from_numpy(Y)


In [11]:
outputs = model(test_X)

In [14]:
outputs.shape

torch.Size([1000, 1])

In [15]:
test_Y

tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
      